In [107]:
LOG_DF_FILE_DIMENSIONS = variables.LOG_DF_FILE_DIMENSIONS
df = pd.read_csv(LOG_DF_FILE_DIMENSIONS, sep="\t")

In [108]:
version_current = max(df["version"])
version_previous = version_current - 1
cond_previous = df["version"] == version_previous
cond_current = df["version"] == version_current
cond_not_binary = df["binary"] == False

In [109]:
version_current

6

In [110]:
version_previous

5

In [111]:
for fn in sorted(df.loc[cond_not_binary, "fn"].unique()):
    cond_fn = df["fn"] == fn
    cond_fnp = cond_previous & cond_fn
    cond_fnc = cond_current & cond_fn
    if sum(cond_fnp) == 0:
        print("{} not present in previous version".format(fn))
    if sum(cond_fnc) == 0:
        print("{} not present in current version".format(fn))
    num_lines_previous = df.loc[cond_fnp, "num_lines"].values[0]
    num_lines_current = df.loc[cond_fnc, "num_lines"].values[0]
    # assert num_lines_current >= num_lines_previous
    try:
        assert num_lines_current >= num_lines_previous
    except AssertionError: # at least 95% of previous
        assert int(100 * num_lines_current / num_lines_previous) >= 95

In [112]:
df.head()

,fn,binary,size,num_lines,date,md5,version
0,Functions_table_UPS_FIN.txt,False,524106423,3521106.0,2020-10-03 12:10:49.197834,82a3c1361fb59bb8b62a33032ab4394d,1
1,KEGG_Taxid_2_acronym_table_UPS_FIN.txt,False,59930,5309.0,2020-10-02 20:11:31.759434,2558a0b572cda1d493e4c62835db546a,1
2,Lineage_table_UPS_FIN.txt,False,4628384,72924.0,2020-10-03 07:34:54.538533,d09f71b11e55b609eb5b8e7615438416,1
3,Protein_2_FunctionEnum_table_UPS_FIN.txt,False,30452367757,171659103.0,2020-10-03 13:06:34.370964,2b26b05ca60e5627fdda5f347bdfc9ed,1
4,Secondary_2_Primary_ID_UPS_FIN.txt,False,6419770008,187401348.0,2020-09-23 08:07:12.554013,eee716ad020225311c830c7aa436cfb3,1


In [113]:
df.tail()

,fn,binary,size,num_lines,date,md5,version
253,indices_arr_UPS_FIN.p,True,14147126,NaN,2020-12-01 10:30:11.868468,138ed1d49b191905ec15d512e4ebd27e,6
254,taxid_2_tuple_funcEnum_index_2_associations_counts_pickle_UPS_FIN.p,True,798403646,NaN,2020-12-01 13:37:46.773188,62b6e3e183d9a82fc858ad7723805a0e,6
255,year_arr_UPS_FIN.p,True,7073644,NaN,2020-12-01 10:30:11.868468,561271b1794ade241ba7849bb7675bd9,6
256,UniProt_NCBI_TaxID_and_TaxName_for_autocomplete_UPS_FIN.txt,False,1026170,19141.0,2020-12-17 21:06:31.290640,264830356194983eaf7100ab32df48d6,5
257,UniProt_NCBI_TaxID_and_TaxName_for_autocomplete_UPS_FIN.txt,False,1026170,19141.0,2020-12-17 21:06:31.290640,264830356194983eaf7100ab32df48d6,6


In [96]:
import query, userinput
from collections import defaultdict

In [86]:
import numpy as np

In [87]:
import random
taxid = random.choice(query.get_taxids()) # read_from_flat_files=True
background = query.get_proteins_of_taxid(taxid)
foreground = random.sample(background, 200)
intensity = [str(ele) for ele in np.random.normal(size=len(background))]

In [91]:
intensity[:3]

['-1.809734578912832', '-0.07231743498728621', '0.34569802304769165']

In [97]:
args_dict = defaultdict(lambda: None)
args_dict.update({'taxid': taxid, 'output_format': 'tsv', 'enrichment_method': 'abundance_correction', 'FDR_cutoff': '1.0'})
# should be "background_intensity" instead of "intensity" but nonetheless this should work
args_dict.update({'foreground': "%0d".join(foreground), 'background': "%0d".join(background), "intensity": "%0d".join(intensity)})
ui = userinput.REST_API_input(pqo=None, args_dict=args_dict)

In [100]:
ui.check_cleanup, ui.check_parse

(True, True)

In [84]:
fn = r"/Users/dblyon/Downloads/UniProt_NCBI_TaxID_and_TaxName_not_clean.txt"
fn_out = r"/Users/dblyon/modules/cpr/agotool/data/PostgreSQL/tables/UniProt_NCBI_TaxID_and_TaxName_for_autocomplete_UPS_FIN.txt"
strings_2_write_list = []
curated_and_sorted = [
    "Homo sapiens (Human), 9606, UP000005640",                      
    "Caenorhabditis elegans, 6239, UP000001940",
    "Drosophila melanogaster (Fruit fly), 7227, UP000000803",
    "Escherichia coli (strain K12), 83333, UP000000625",
    "Mus musculus (Mouse), 10090, UP000000589",
    "Rattus norvegicus (Rat), 10116, UP000002494",
    "Saccharomyces cerevisiae (strain ATCC 204508 / S288c) (Baker's yeast), 559292, UP000002311", # 4932
    "Schizosaccharomyces pombe (strain 972 / ATCC 24843) (Fission yeast), 284812, UP000002485", # 4896
    "Arabidopsis thaliana (Mouse-ear cress), 3702, UP000006548"]
with open(fn, "r") as fh_in:
    for line in fh_in:
        if line.strip() == "Proteome_ID	Tax_ID	OSCODE	SUPERREGNUM	#(1)	#(2)	#(3)	Species Name":
            break
    for line in fh_in:
        if line.startswith("UP"):
            Proteome_ID, Tax_ID, OSCODE, SUPERREGNUM, num_1, num_2, num_3, SpeciesName = line.split("\t")
            strings_2_write_list.append("{}, {}, {}".format(SpeciesName.strip(), Tax_ID, Proteome_ID))
with open(fn_out, "w") as fh_out:
    strings_2_write_list = sorted(strings_2_write_list)
    strings_2_write_list = curated_and_sorted + sorted(set(strings_2_write_list) - set(curated_and_sorted))
    for ele in strings_2_write_list:
        fh_out.write(ele + "\n")

In [83]:
strings_2_write_list[:3]

['Homo sapiens (Human), 9606, UP000005640',
 'Caenorhabditis elegans, 6239, UP000001940',
 'Drosophila melanogaster (Fruit fly), 7227, UP000000803']

In [78]:
l = curated_and_sorted + sorted(set(strings_2_write_list) - set(curated_and_sorted))

In [79]:
l[:15]

['Homo sapiens (Human), 9606, UP000005640',
 'Caenorhabditis elegans, 6239, UP000001940',
 'Drosophila melanogaster (Fruit fly), 7227, UP000000803',
 'Escherichia coli (strain K12), 83333, UP000000625',
 'Mus musculus (Mouse), 10090, UP000000589',
 'Rattus norvegicus (Rat), 10116, UP000002494',
 "Saccharomyces cerevisiae (strain ATCC 204508 / S288c) (Baker's yeast), 559292, UP000002311",
 'Schizosaccharomyces pombe (strain 972 / ATCC 24843) (Fission yeast), 284812, UP000002485',
 'Arabidopsis thaliana (Mouse-ear cress), 3702, UP000006548',
 "'Echinacea purpurea' witches'-broom phytoplasma, 1123738, UP000050566",
 '229E-related bat coronavirus, 1739614, UP000096398',
 'ANME-2 cluster archaeon, 2056317, UP000248329',
 'ANMV-1 virus, 1605378, UP000241134',
 'Abaca bunchy top virus, 438782, UP000202363',
 'Abalone herpesvirus (isolate Abalone/Australia/Victoria/2009) (AbHV), 1241371, UP000029777']

In [53]:
import taxonomy
ncbi = taxonomy.NCBI_taxonomy(taxdump_directory=variables.DOWNLOADS_DIR, for_SQL=False, update=True)


Downloading: ftp://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump.tar.gz
TO: /Users/dblyon/modules/cpr/agotool/data/PostgreSQL/downloads/taxdump.tar.gz
finished download
Extracting files and removing .tar.gz
tar -zxvf /Users/dblyon/modules/cpr/agotool/data/PostgreSQL/downloads/taxdump.tar.gz -C /Users/dblyon/modules/cpr/agotool/data/PostgreSQL/downloads


In [60]:
ncbi.get_sciname(9606)

'Homo sapiens'

In [28]:
pwd

'/Users/dblyon/modules/cpr/agotool/app/python'

In [48]:
import variables, plot_and_table
import pandas as pd
from importlib import reload

In [49]:
reload(plot_and_table)
df = pd.read_csv(variables.fn_example, sep="\t")
df, category_rank_arr = plot_and_table.ready_df_for_plot(df, lineage_dict=None, enrichment_method="genome")

In [51]:
df.groupby("etype").head(2)

,term,hierarchical_level,description,year,over_under,p_value,FDR,effect_size,ratio_in_foreground,ratio_in_background,...,etype,logFDR,FG_count_2_circle_size,rank_2_transparency,color,category_rank,text_label,marker_line_width,marker_line_color,opacity
0,KW-0539,2,Nucleus,-1,o,0.000006,0.000934,0.145815,0.421053,0.275237,...,-51,6.976356,9.311362,1.0,#1b9e77,0,,1,white,0.7
1,KW-0472,2,Membrane,-1,u,0.000005,0.000934,-0.087144,0.134599,0.221743,...,-51,6.976356,2.976583,0.9,#1b9e77,0,,1,white,0.7
12,GOCC:0005634,8,Nucleus,-1,o,0.000004,0.009358,0.133736,0.522002,0.388266,...,-20,4.671471,11.543799,1.0,#d95f02,1,,1,white,0.7
13,GOCC:0070013,7,Intracellular organelle lumen,-1,o,0.000004,0.009358,0.125108,0.389991,0.264884,...,-20,4.671471,8.624458,0.9,#d95f02,1,,1,white,0.7
30,GO:0070013,5,intracellular organelle lumen,-1,o,0.000005,0.001667,0.130285,0.358067,0.227783,...,-22,6.396531,7.918474,1.0,#7570b3,2,,1,white,0.7
31,GO:0005634,6,nucleus,-1,o,0.000005,0.001667,0.128559,0.530630,0.402071,...,-22,6.396531,11.734605,0.9,#7570b3,2,,1,white,0.7


In [52]:
df.category.unique()

[UniProt keywords, GO cellular component TextMining, GO cellular component]
Categories (3, object): [UniProt keywords, GO cellular component TextMining, GO cellular component]

In [38]:
df.head()

,term,hierarchical_level,description,year,over_under,p_value,FDR,effect_size,ratio_in_foreground,ratio_in_background,...,etype,logFDR,FG_count_2_circle_size,rank_2_transparency,color,text_label,marker_line_width,marker_line_color,opacity,category_rank
0,KW-0539,2,Nucleus,-1,o,0.000006,0.000934,0.145815,0.421053,0.275237,...,-51,6.976356,9.311362,1.0,#1b9e77,,1,white,0.7,41
1,KW-0472,2,Membrane,-1,u,0.000005,0.000934,-0.087144,0.134599,0.221743,...,-51,6.976356,2.976583,0.9,#1b9e77,,1,white,0.7,42
2,KW-0805,3,Transcription regulation,-1,o,0.000002,0.000934,0.075928,0.123382,0.047455,...,-51,6.976356,2.728534,0.8,#1b9e77,,1,white,0.7,43
3,KW-0804,2,Transcription,-1,o,0.000003,0.000934,0.073339,0.133736,0.060397,...,-51,6.976356,2.957502,0.7,#1b9e77,,1,white,0.7,44
4,KW-0812,3,Transmembrane,-1,u,0.000003,0.000934,-0.071613,0.059534,0.131148,...,-51,6.976356,2.000000,0.6,#1b9e77,,1,white,0.7,45


In [46]:
df['category_rank'] = df["category"].cat.codes

,term,hierarchical_level,description,year,over_under,p_value,FDR,effect_size,ratio_in_foreground,ratio_in_background,...,etype,logFDR,FG_count_2_circle_size,rank_2_transparency,color,text_label,marker_line_width,marker_line_color,opacity,category_rank
0,KW-0539,2,Nucleus,-1,o,0.000006,0.000934,0.145815,0.421053,0.275237,...,-51,6.976356,9.311362,1.0,#1b9e77,,1,white,0.7,0
1,KW-0472,2,Membrane,-1,u,0.000005,0.000934,-0.087144,0.134599,0.221743,...,-51,6.976356,2.976583,0.9,#1b9e77,,1,white,0.7,0
12,GOCC:0005634,8,Nucleus,-1,o,0.000004,0.009358,0.133736,0.522002,0.388266,...,-20,4.671471,11.543799,1.0,#d95f02,,1,white,0.7,1
13,GOCC:0070013,7,Intracellular organelle lumen,-1,o,0.000004,0.009358,0.125108,0.389991,0.264884,...,-20,4.671471,8.624458,0.9,#d95f02,,1,white,0.7,1
30,GO:0070013,5,intracellular organelle lumen,-1,o,0.000005,0.001667,0.130285,0.358067,0.227783,...,-22,6.396531,7.918474,1.0,#7570b3,,1,white,0.7,2
31,GO:0005634,6,nucleus,-1,o,0.000005,0.001667,0.128559,0.530630,0.402071,...,-22,6.396531,11.734605,0.9,#7570b3,,1,white,0.7,2


In [29]:
cd app/python

[Errno 2] No such file or directory: 'app/python'
/Users/dblyon/modules/cpr/agotool/app/python


In [4]:
import variables

In [31]:
import igraph as ig
# import plotly.plotly as py
from plotly.graph_objs import *

import os, sys, json
import datetime
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 300) # in order to prevent 50 character cutoff of to_html export / ellipsis
# from plotly_tools import data_bars
import dash
from dash.dependencies import Input, Output, State
import dash_table
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
import dash_daq as daq
# import plotly.express as px
import plotly.graph_objects as go

ModuleNotFoundError: No module named 'igraph'

In [107]:
from itertools import combinations
from collections import defaultdict

# ToDo: term_2_connectedTerms_dict: key=term, val=set of terms --> Nodes
def get_term_2_edges_dict(df, lineage_dict_direct):
    """
    iterate over direct parents and stop once at least one is found
    Dict: key=term val=[X_vals_list, Y_vals_list, Weights_list]
    # for every term:
    #   for every edge:
    #     e.g. between point A (Ax, Ay) and B (Bx, By) as well as A (Ax, Ay) and C (Cx, Cy)
    #     X_point_list += [Ax, Bx, None]
    #     Y_point_list += [Ay, By, None]
    #     Weights_list += [A_2_B]
    #     X_point_list += [Ax, Cx, None]
    #     Y_point_list += [Ay, Cy, None]    
    """
    term_2_edges_dict = defaultdict(lambda: {"X_points": [], "Y_points": [], "Weights": [], "Nodes": []})
    for etype, group in df.groupby("etype"):
        if etype not in variables.entity_types_with_ontology:
            continue
        funcNameFamilySet_list_merged, funcNameFamilySet_list_temp, edgesXYCoords_list, funcNamePair_set = [], [], [], set()
        funcs_2_travers_set = group["term"]
        for term in funcs_2_travers_set:
            funcNamePair_set |= find_pairs_2_link(term, lineage_dict_direct, funcs_2_travers_set)
                
        ## extract x and y coordinates and a weight depending on the overlapp of FG_IDs (Jaccard index)
        for funcNamePair in funcNamePair_set:
            X_points_list, Y_points_list, Weight_temp = get_edge_positions_and_weight_as_list_v3(group[group["term"].isin(funcNamePair)])
            funcName_a, funcName_b = funcNamePair
            term_2_edges_dict[funcName_a]["X_points"] += X_points_list
            term_2_edges_dict[funcName_a]["Y_points"] += Y_points_list
            term_2_edges_dict[funcName_a]["Weights"].append(Weight_temp)
            term_2_edges_dict[funcName_a]["Nodes"].append(funcName_b)
            
            term_2_edges_dict[funcName_b]["X_points"] += X_points_list
            term_2_edges_dict[funcName_b]["Y_points"] += Y_points_list
            term_2_edges_dict[funcName_b]["Weights"].append(Weight_temp)
            term_2_edges_dict[funcName_b]["Nodes"].append(funcName_a)
    return term_2_edges_dict     

def find_pairs_2_link(term, lineage_dict_direct, funcs_2_travers_set):
    funcNamePair_set = set()
    for parents in yield_direct_parents([term], lineage_dict_direct): 
        if not parents:
            continue
        else:
            lineage_2_add = set(parents)
            lineage_2_add.add(term)
            funcNameFamilySet = lineage_2_add.intersection(funcs_2_travers_set)
        len_funcNameFamilySet = len(funcNameFamilySet)
        if len_funcNameFamilySet < 2:
            continue
        elif len_funcNameFamilySet == 2:
            funcNamePair_set.add(tuple(sorted(funcNameFamilySet)))
            return funcNamePair_set
        elif len_funcNameFamilySet > 2:
            for func in funcNameFamilySet - set([term]):
                funcNamePair_set.add(tuple(sorted([term, func])))
            return funcNamePair_set
        else:
            raise StopIteration
    return funcNamePair_set

def get_edge_positions_and_weight_as_list_v3(df):
    """
    # for every term:
    #   for every edge:
    #     e.g. between point A (Ax, Ay) and B (Bx, By) as well as A (Ax, Ay) and C (Cx, Cy)
    #     X_point_list += [Ax, Bx, None]
    #     Y_point_list += [Ay, By, None]
    #     Weights_list += [A_2_B]
    #     X_point_list += [Ax, Cx, None]
    #     Y_point_list += [Ay, Cy, None]
    """
    assert df.shape[0] == 2
    
    try:
        first_FG_IDs, second_FG_IDs = df["FG_IDs"].values
        first_FG_IDs = set(first_FG_IDs.split(";"))
        second_FG_IDs = set(first_FG_IDs.split(";"))
        ## compute weight of edge
        jaccard_index = len(first_FG_IDs.intersection(second_FG_IDs)) / len(first_FG_IDs.union(second_FG_IDs))
    except: #  AttributeError if NaN or ZeroDivisionError if ... 
        jaccard_index = 1
    return df["logFDR"].tolist() + [None], df["effectSize"].tolist() + [None], jaccard_index

def yield_direct_parents(terms, lineage_dict):
    """
    generator: to return all direct parents of given terms
    lineage_dict = cst.get_lineage_dict_for_all_entity_types_with_ontologies(direct_or_allParents="direct")
    terms = ["GO:0016572"]
    for parents in yield_direct_parents(terms, lineage_dict):
        print(parents)
    :param terms: list of string
    :param lineage_dict: child 2 direct parents dict
    :return: list of string
    """
    def get_direct_parents(terms, lineage_dict):
        parents = []
        for term in terms:
            try:
                parents += lineage_dict[term]
            except KeyError:
                pass
        parents = sorted(set(parents))
        return parents
    parents = get_direct_parents(terms, lineage_dict)
    while parents:
        yield parents
        parents = get_direct_parents(parents, lineage_dict)
    return None

# def find_edges_via_hierarchy_v3(df, lineage_dict_direct):
#     """
#     for each etype find edges to draw in plot of x="logFDR", y="effectSize"
#     add funcEnum and lineage to set
#     v1: merge sets if there is overlapp --> wrong since everything gets connected
#     v2: don't merge set, simply check if >= 2 funcEnums from given user data present
#     filter by funcEnums that are in the given user data
#     v3: iterate over direct parents and stop once at least one is found
#     e.g. of return
#     {-57: [],
#      -51: [{47939, 47977, 48117, 48335, 48487}],
#      ...}     
#     df: DataFrame of aGOtool output with ['funcEnum', 'etype'] cols
#     lineage_dict_enum: key: function enumeration, value: set of func enum array all parents
#     :return: etype_2_funcEnumFamilySet_list_dict, key: etype(int), val: list of sets (int)
#     """
#     x_startL, x_stopL, y_startL, y_stopL, weightL, etypeL = [], [], [], [], [], []
#     etype_2_funcNamePair_set_dict, etype_2_edgesXYCoords_list_dict = {}, {} 
#     for etype, group in df.groupby("etype"):
#         funcNameFamilySet_list_merged, funcNameFamilySet_list_temp, edgesXYCoords_list, funcNamePair_set = [], [], [], set()
#         if etype not in variables.entity_types_with_ontology:
#             continue
#         funcs_2_travers_set = group["term"]
#         for term in funcs_2_travers_set:
#             funcNamePair_set |= find_pairs_2_link(term, lineage_dict_direct, funcs_2_travers_set)
                
#         ## extract x and y coordinates and a weight depending on the overlapp of FG_IDs (Jaccard index)
#         for funcNamePair in funcNamePair_set:
#             t = get_edge_positions_and_weight_as_list(group[group["term"].isin(funcNamePair)])
#             x_startL_2merge, x_stopL_2merge, y_startL_2merge, y_stopL_2merge, weightL_2merge = t
#             x_startL += x_startL_2merge
#             x_stopL += x_stopL_2merge
#             y_startL += y_startL_2merge
#             y_stopL += y_stopL_2merge
#             weightL += weightL_2merge
#             etypeL += len(weightL_2merge) * [etype]                                        
#         etype_2_funcNamePair_set_dict[etype] = funcNamePair_set
#     df_coords = pd.DataFrame()
#     df_coords["etype"] = etypeL
#     df_coords["x_start"] = x_startL
#     df_coords["x_stop"] = x_stopL    
#     df_coords["y_start"] = y_startL
#     df_coords["y_stop"] = y_stopL
#     df_coords["weight"] = weightL        
#     return etype_2_funcNamePair_set_dict, df_coords #etype_2_edgesXYCoords_list_dict
# 
# def get_edge_positions_and_weight_as_list_v2(df):
#     """
#     # for every term:
#     #   for every edge:
#     #     e.g. between point A (Ax, Ay) and B (Bx, By) as well as A (Ax, Ay) and C (Cx, Cy)
#     #     X_point_list += [Ax, Bx, None]
#     #     Y_point_list += [Ay, By, None]
#     #     Weights_list += [A_2_B]
#     #     X_point_list += [Ax, Cx, None]
#     #     Y_point_list += [Ay, Cy, None]
#     """
#     X_point_list, Y_point_list, Weights_list = [], [], []
#     for first_index, second_index in combinations(df.index, 2):
#         try:
#             first_FG_IDs = set(df.loc[first_index, "FG_IDs"].split(";"))
#             second_FG_IDs = set(df.loc[second_index, "FG_IDs"].split(";"))
#             ## compute weight of edge
#             jaccard_index = len(first_FG_IDs.intersection(second_FG_IDs)) / len(first_FG_IDs.union(second_FG_IDs))
#         except: #  AttributeError if NaN or ZeroDivisionError if ... 
#             jaccard_index = 1
#         X_points_list += df.loc[[first_index, second_index], "logFDR"].tolist() + [None]
#         Y_points_list += df.loc[[first_index, second_index], "effectSize"].tolist() + [None]
#         Weights_list.append(jaccard_index)
#     return X_points_list, Y_points_list, Weights_list

In [1]:
df.head()

NameError: name 'df' is not defined

In [3]:
cd app/python

/Users/dblyon/modules/cpr/agotool/app/python


In [6]:
import create_SQL_tables_snakemake as cst
import variables

In [5]:
lineage_dict = cst.get_lineage_dict_for_all_entity_types_with_ontologies(direct_or_allParents="direct")

In [10]:
# lineage_dict

In [8]:
import pickle, os
pickle.dump(lineage_dict, open(os.path.join(variables.TABLES_DIR, "lineage_dict_direct.p"), "wb"))

In [108]:
term_2_edges_dict = get_term_2_edges_dict(df, lineage_dict)

In [109]:
term_2_edges_dict["KW-0539"]

{'X_points': [], 'Y_points': [], 'Weights': [], 'Nodes': []}

In [11]:
pwd

'/Users/dblyon/modules/cpr/agotool/app/python'

In [26]:
fn = os.path.join(variables.TABLES_DIR, "aGOtool_flatfiles_current.tar.gz")
import tools
import datetime
t = tools.creation_date(fn)
mtime = datetime.datetime.fromtimestamp(t).strftime("%d %B %Y")
mtime

'01 December 2020'

In [14]:
import pathlib
fn = os.path.join(variables.TABLES_DIR, "aGOtool_flatfiles_current.tar.gz")
fname = pathlib.Path(fn)
assert fname.exists(), f'No such file: {fname}'  # check that the file exists
print(fname.stat())

os.stat_result(st_mode=33188, st_ino=96348834, st_dev=16777222, st_nlink=1, st_uid=501, st_gid=20, st_size=7385282021, st_atime=1608019987, st_mtime=1606827655, st_ctime=1608020325)


In [17]:
import datetime
mtime = datetime.datetime.fromtimestamp(fname.stat().st_mtime)
mtime

datetime.datetime(2020, 12, 1, 14, 0, 55, 620368)

In [19]:
import time
created = os.path.getctime(fn)
print("Date created: "+time.ctime(created))
print("Date created:",datetime.datetime.fromtimestamp(created))

Date created: Tue Dec 15 09:18:45 2020
Date created: 2020-12-15 09:18:45.678633


In [93]:
pwd

'/Users/dblyon/modules/cpr/agotool/app/python'

In [101]:
term_2_edges_dict = dict(term_2_edges_dict)

In [102]:
import pickle
pickle.dump(term_2_edges_dict, open("term_2_edges_dict.p", "wb"))

In [86]:
# for term in term_2_edges_dict:
#     for weight in term_2_edges_dict[term]["Weights"]:
#         if weight < 1:
#             print(term, weight)

In [56]:
df = pd.read_csv(variables.fn_example, sep="\t")
etype_2_funcNamePair_set_dict, etype_2_edgesXYCoords_list_dict = find_edges_via_hierarchy_v3(df, lineage_dict)

In [87]:
# df.head()

In [72]:
first_index=0
second_index=4
df.loc[[first_index, second_index], "logFDR"].tolist() + [None]

[4.671470586861855, 4.671470586861855, None]

In [77]:
x = df.loc[[first_index, second_index], ]
x

,term,hierarchical_level,description,year,over_under,p_value,FDR,effectSize,ratio_in_FG,ratio_in_BG,...,FG_IDs,s_value,rank,funcEnum,category,etype,logFDR,FG_count_2_circle_size,rank_2_transparency,color
0,GOCC:0005634,8,Nucleus,-1,o,0.000004,0.009358,0.133736,0.522002,0.388266,...,P31383;P37898;P14164;P40535;P31787;Q08981;P52910;P60010;Q02336;P53909;Q2V2Q1;P47143;Q01976;P07248;Q12433;Q12156;P10869;P47019;P16550;P38281;P32454;P53940;P22936;P40024;Q04371;P08566;P32449;P80428;P53946;Q05123;P32447;P41696;P22035;Q12186;P47176;Q06338;P35817;Q06631;P43583;P29311;P34730;Q08965;P4...,0.716327,1,254,Gene Ontology cellular component TEXTMINING,-20,4.671471,11.543799,1.0,#1b9e77
4,GOCC:0043232,7,Intracellular non-membrane-bounded organelle,-1,o,0.000017,0.009358,0.088870,0.459016,0.370147,...,P31383;Q02486;P15891;P40535;Q08981;P19414;P52910;P60010;Q04894;P38266;Q12156;P10869;P46367;P46672;P11076;P32449;P47117;P80428;P53946;Q05123;Q05933;P32447;P07251;P00830;P22035;P35817;Q06631;P29311;Q08965;P40493;Q07457;P53741;Q04347;P25627;P23293;P06787;P28495;P34237;P17106;P32504;P33322;P21560;P0...,0.424281,5,1879,Gene Ontology cellular component TEXTMINING,-20,4.671471,10.150911,0.6,#1b9e77


In [78]:
x["FG_IDs"]

0    P31383;P37898;P14164;P40535;P31787;Q08981;P52910;P60010;Q02336;P53909;Q2V2Q1;P47143;Q01976;P07248;Q12433;Q12156;P10869;P47019;P16550;P38281;P32454;P53940;P22936;P40024;Q04371;P08566;P32449;P80428;P53946;Q05123;P32447;P41696;P22035;Q12186;P47176;Q06338;P35817;Q06631;P43583;P29311;P34730;Q08965;P4...
4    P31383;Q02486;P15891;P40535;Q08981;P19414;P52910;P60010;Q04894;P38266;Q12156;P10869;P46367;P46672;P11076;P32449;P47117;P80428;P53946;Q05123;Q05933;P32447;P07251;P00830;P22035;P35817;Q06631;P29311;Q08965;P40493;Q07457;P53741;Q04347;P25627;P23293;P06787;P28495;P34237;P17106;P32504;P33322;P21560;P0...
Name: FG_IDs, dtype: object

In [58]:
etype_2_edgesXYCoords_list_dict

,etype,x_start,x_stop,y_start,y_stop,weight
0,-51,6.976356,6.976356,-0.087144,-0.070751,0.435897
1,-51,6.976356,6.976356,-0.071613,-0.070751,0.985507
2,-51,6.976356,6.976356,0.075928,0.073339,0.922581
3,-51,6.976356,6.976356,-0.087144,-0.071613,0.442308
4,-22,6.396531,6.396531,0.124245,0.047455,0.330065
...,...,...,...,...,...,...
94,-20,4.671471,4.671471,0.125108,0.057808,0.309735
95,-20,4.671471,3.641455,0.059534,0.068162,0.172193
96,-20,4.671471,4.671471,0.088007,0.064711,0.341418
97,-20,4.671471,4.671471,0.125108,0.044003,0.306267


In [63]:
for parents in yield_direct_parents(["GO:0000001"], lineage_dict):
    print(parents)

['GO:0048308', 'GO:0048311']
['GO:0006996', 'GO:0007005', 'GO:0051646']
['GO:0006996', 'GO:0016043', 'GO:0051640']
['GO:0016043', 'GO:0051641', 'GO:0071840']
['GO:0009987', 'GO:0051179', 'GO:0071840']
['GO:0008150', 'GO:0009987']
['GO:0008150']


In [65]:
lineage_dict['GO:0048311']

['GO:0007005', 'GO:0051646']